# Simulación y Análisis de Métricas SDN con gRPC (Dispositivo Único)

Este cuaderno simula un único dispositivo de red (switch) y su gestión mediante una interfaz Northbound basada en **gRPC**.

Incluye la generación de métricas de CPU y tráfico, la simulación de un fallo, y la visualización de los datos recolectados.

---

### Metodo 1 - Configuración Manual (Recomendado para un entorno limpio)

1. **Crea un entorno virtual** y asegúrate de haber instalado las dependencias desde el archivo `requirements.txt` ubicado en la raíz del proyecto:

   ```bash
   pip install -r ../requirements.txt  # Si ejecutas desde src
   # o, si copiaste requirements.txt a src/
   pip install -r requirements.txt

2. **Genera los stubs de gRPC** si los archivos `sdn_northbound_pb2.py` y `sdn_northbound_pb2_grpc.py` no están presentes en la carpeta `src/`.

   Puedes hacerlo ejecutando la siguiente celda en tu cuaderno o directamente desde la terminal (asegúrate de estar ubicado dentro de la carpeta `src/`):

   ```bash
   python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. sdn_northbound.proto

### Metodo 2 - Ejecución Directa desde los Jupyter Notebooks (Menos recomendado para producción)*ç

1. **Instalar dependencias** (si no están ya).
   ```bash
   pip install grpcio-tools matplotlib numpy seaborn

2. **Crear el archivo .proto**
   ```bash
   with open('sdn_northbound.proto', 'w') as f:
      f.write(""" ...contenido del proto... """)

3. **Generar los stubs gRPC**
   ```bash
   !python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. sdn_northbound.proto

**Nota**: todos estos pasos ya estan en los Notebooks, solo hace falta descomentarlos.

In [1]:
# P1. Instalar dependencias
#!pip install grpcio-tools matplotlib numpy seaborn

# P2. Crear archivo .proto
#with open('sdn_northbound.proto', 'w') as f:
#    f.write("""
#syntax = "proto3";

#service SDNNorthbound {
#  rpc UpdateVlan(VlanConfig) returns (ConfigResponse) {}
#  rpc StreamMetrics(DeviceQuery) returns (stream MetricData) {}
#  rpc ReportFault(FaultNotification) returns (FaultAck) {}
#}

#message VlanConfig {
#  string device_id = 1;
#  uint32 vlan_id = 2;
#  string name = 3;
#}

#message ConfigResponse {
#  bool success = 1;
#  string message = 2;
#}

#message DeviceQuery {
#  string device_id = 1;
#  uint32 interval = 2;
#}

#message MetricData {
#  float cpu = 1;
#  uint64 traffic = 2;
#  string timestamp = 3;
#}

#message FaultNotification {
#  string device_id = 1;
#  string description = 2;
#}

#message FaultAck {
#  bool received = 1;
#  string timestamp = 2;
#}
#""")


# P3. Generar código gRPC
#!python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. sdn_northbound.proto

## Paso 4: Implementar Servidor gRPC con Topología Compleja
import grpc
from concurrent import futures
import time
import numpy as np
from datetime import datetime
from sdn_northbound_pb2 import *
from sdn_northbound_pb2_grpc import *

class NetworkTopology:
    def __init__(self):
        self.devices = {
            "core_switch": {"vlans": [], "traffic": 242000, "cpu": 40, "status": "up"},
            "access_switch1": {"vlans": [], "traffic": 120000, "cpu": 30, "status": "up"},
            "router1": {"interfaces": ["Gig0/1", "Gig0/2"], "cpu": 30, "traffic": 50000},
            "firewall1": {"rules": ["default-deny"], "status": "active", "traffic": 15000}
        }

class NorthboundServicer(SDNNorthboundServicer):
    def __init__(self):
        self.topology = NetworkTopology()
        self.fault_log = []

    def UpdateVlan(self, request, context):
        device = request.device_id
        if device in self.topology.devices and "vlans" in self.topology.devices[device]:
            self.topology.devices[device]["vlans"].append(request.vlan_id)
            return ConfigResponse(
                success=True,
                message=f"VLAN {request.vlan_id} agregada a {device}. VLANs actuales: {self.topology.devices[device]['vlans']}"
            )
        return ConfigResponse(success=False, message="Dispositivo no soporta VLANs")

    def StreamMetrics(self, request, context):
        device = request.device_id
        for _ in range(10):  # Simular 10 intervalos
            # Generar métricas realistas con fluctuaciones
            base_traffic = self.topology.devices[device]["traffic"]
            traffic = base_traffic * (1 + 0.3 * np.random.pareto(2))
            cpu = 40 + (traffic / 500000) * 60 + np.random.normal(0, 5)

            # Detectar fallo si CPU > 95%
            if cpu > 95 and device not in self.fault_log:
                self.fault_log.append(device)
                self.ReportFaultInternal(f"CPU crítica en {device}")

            yield MetricData(
                cpu=np.clip(round(cpu, 2), 0, 100),
                traffic=int(traffic),
                timestamp=datetime.now().isoformat()
            )
            time.sleep(request.interval)

    def ReportFault(self, request, context):
        self.topology.devices[request.device_id]["status"] = "down"
        return FaultAck(
            received=True,
            timestamp=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        )

    def ReportFaultInternal(self, description):
        print(f"🔴 [Sistema] {description} - Acción: Aislar dispositivo")

def start_server():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    add_SDNNorthboundServicer_to_server(NorthboundServicer(), server)
    server.add_insecure_port('[::]:50051')
    server.start()
    print("🟢 Servidor iniciado - Topología: Core-Access (3 capas)")
    return server

## Paso 5: Cliente Avanzado con Gestión de Topología
def test_client():
    channel = grpc.insecure_channel('localhost:50051')
    stub = SDNNorthboundStub(channel)

    # Operaciones extendidas
    devices_to_configure = [
        ("access_switch1", 100, "VLAN_Mgmt"),
        ("core_switch", 200, "VLAN_Data")
    ]

    for device, vlan, name in devices_to_configure:
        response = stub.UpdateVlan(VlanConfig(
            device_id=device,
            vlan_id=vlan,
            name=name
        ))
        print(f"🔧 Configuración {device}: {response.message}")

    # Monitoreo multi-dispositivo
    devices_to_monitor = ["core_switch", "access_switch1", "router1"]
    for device in devices_to_monitor:
        print(f"\n📊 Métricas en vivo - {device}:")
        metrics = stub.StreamMetrics(DeviceQuery(device_id=device, interval=1))
        try:
            for i, metric in enumerate(metrics):
                print(f"  ⏱️ {datetime.fromisoformat(metric.timestamp).strftime('%H:%M:%S')} | CPU: {metric.cpu}% | Tráfico: {metric.traffic:,} bps")
                if i >= 5:  # Limitar a 5 lecturas por dispositivo
                    break
        except Exception as e:
            print(f"⚠️ Error en {device}: {str(e)}")

    # Simular fallo
    ack = stub.ReportFault(FaultNotification(
        device_id="firewall1",
        description="Bloqueo de tráfico no autorizado"
    ))
    print(f"\n🚨 Estado firewall: {ack.timestamp} - ACK recibido")

## Paso 6: Ejecución con Gestión de Topología
import threading

server = start_server()
threading.Timer(2, test_client).start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    server.stop(0)
    print("\n🔴 Servidor detenido - Estado final de la topología:")
    print(NorthboundServicer().topology.devices)

🟢 Servidor iniciado - Topología: Core-Access (3 capas)
🔧 Configuración access_switch1: VLAN 100 agregada a access_switch1. VLANs actuales: [100]
🔧 Configuración core_switch: VLAN 200 agregada a core_switch. VLANs actuales: [200]

📊 Métricas en vivo - core_switch:
  ⏱️ 12:07:46 | CPU: 71.86000061035156% | Tráfico: 305,753 bps
  ⏱️ 12:07:47 | CPU: 90.62000274658203% | Tráfico: 365,420 bps
  ⏱️ 12:07:48 | CPU: 90.08000183105469% | Tráfico: 366,430 bps
  ⏱️ 12:07:49 | CPU: 77.80000305175781% | Tráfico: 246,325 bps
🔴 [Sistema] CPU crítica en core_switch - Acción: Aislar dispositivo
  ⏱️ 12:07:50 | CPU: 100.0% | Tráfico: 2,344,210 bps
  ⏱️ 12:07:51 | CPU: 75.97000122070312% | Tráfico: 271,089 bps

📊 Métricas en vivo - access_switch1:
  ⏱️ 12:07:51 | CPU: 51.349998474121094% | Tráfico: 121,499 bps
  ⏱️ 12:07:52 | CPU: 64.19000244140625% | Tráfico: 188,797 bps
  ⏱️ 12:07:53 | CPU: 55.970001220703125% | Tráfico: 156,962 bps
  ⏱️ 12:07:54 | CPU: 44.72999954223633% | Tráfico: 129,142 bps
  ⏱️ 12: